**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
sudo docker run --rm -v <ruta a mi directorio>:/datos --cap-add SYS_ADMIN --privileged --name jupyterlab-numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Nota generada a partir de [liga](https://www.dropbox.com/s/fyqwiqasqaa3wlt/3.1.1.Multiplicacion_de_matrices_y_estructura_de_datos.pdf?dl=0), [liga2](https://www.dropbox.com/s/l4hq45rj860ql9f/3.1.2.Localidad_y_vectorizacion.Analisis_del_error_en_computos_matriciales_basicos.pdf?dl=0)

# 3.1 El cómputo matricial y el álgebra lineal. Vectorización, BLAS y el uso del caché eficientemente.

El cómputo matricial está construído sobre una jerarquía de operaciones del álgebra lineal:

* Productos punto involucran operaciones escalares de suma y multiplicación (nivel BLAS 1).

* La multiplicación matriz-vector está hecha de productos punto (nivel BLAS 2).

* La multiplicación matriz-matriz utiliza colecciones de productos matriz-vector (nivel BLAS 3).

Las operaciones anteriores se describen en el álgebra lineal con la teoría de espacios vectoriales pero también es posible describirlas en una forma algorítmica. Ambas descripciones se complementan una a la otra.

Manejaremos nombres que en el [Linear Algebra Package: LAPACK](http://www.netlib.org/lapack/explore-html/dir_fa94b7b114d387a7a8beb2e3e22bf78d.html) son utilizados para denotar algunas operaciones con escalares, vectores o matrices. Ver [ Reference-LAPACK / lapack](https://github.com/Reference-LAPACK/lapack) para su github.

## Operación del producto interno estándar o producto punto

Consideramos $x,y \in \mathbb{R}^n$. El producto punto entre $x$ y $y$ es $c = x^Ty = \displaystyle \sum_{i=1}^n x_iy_i$. 

**Ejemplo y algoritmo del producto punto:**

In [2]:
c=0
n=5
x=[-1]*n
y=[1.5]*n

for i in range(n):
    c += x[i]*y[i]

In [3]:
c

-7.5

**Obs:**

* El producto punto de dos $n$-vectores involucran $n$ multiplicaciones y $n$ sumas para un total de $2n$ operaciones. Usamos la notación $\mathcal{O}(\cdot)$ para escribir que el producto punto es $\mathcal{O}(n)$ y se lee "de orden $n$ o proporcional a $n$" para indicar que la **cantidad de trabajo** tiene un comportamiento **lineal** con la dimensión $n$. También tal cantidad de trabajo opera sobre una **cantidad lineal de datos**.

## Operación **saxpy**

Consideramos $\alpha \in \mathbb{R}, x,y \in \mathbb{R}^n$. El nombre lo recibe por *scalar alpha x plus y*. En LAPACK se escribe en forma *update*:

$$y=\alpha x + y \therefore y_i = \alpha x_i + y_i \forall i=1,...,n$$

**Obs:** 

* El símbolo $=$ no se utiliza como igualdad de expresiones sino como en computación para denotar asignación.

* También encontramos en LAPACK `caxpy` o `daxpy` para el caso complejo y para números en doble precisión respectivamente.

* Ésta operación realiza un trabajo de $\mathcal{O}(n)$ sobre una cantidad de datos $\mathcal{O}(n)$.

**Ejemplo y algoritmo de saxpy:**

In [4]:
alpha=2
n=5
x=[-2]*n
y=[0]*n

for i in range(n):
    y[i] += alpha*x[i]

In [5]:
y

[-4, -4, -4, -4, -4]

o en una forma *update*:

In [6]:
alpha=2
n=5
x=[-2]*n
y=[3,4,-1,0,1]

for i in range(n):
    y[i] += alpha*x[i]

In [7]:
y

[-1, 0, -5, -4, -3]

**Comentario:** La operación de producto punto y *saxpy* son algoritmos catalogados como de **nivel BLAS 1** (ver [BLAS: Basic Linear Algebra Subprograms](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms). Éstos algoritmos se caracterizan por involucrar una cantidad de trabajo lineal sobre una cantidad lineal de datos. Ver [level 1](http://www.netlib.org/blas/#_level_1) para más ejemplos.

## Multiplicación matriz-vector

Consideramos $A \in \mathbb{R}^{m \times n}, x \in \mathbb{R}^n, y \in \mathbb{R}^m$. La operación $y = y + Ax$ es una operación *generalizada* saxpy, por ello se denomina **gaxpy** pero en LAPACK podemos encontrar esta operación con nombres como [sgemv](http://www.netlib.org/lapack/explore-html/db/d58/sgemv_8f.html), [dgemv](http://www.netlib.org/lapack/explore-html/dc/da8/dgemv_8f.html), [cgemv](http://www.netlib.org/lapack/explore-html/d4/d8a/cgemv_8f.html) o [zgemv](http://www.netlib.org/lapack/explore-html/db/d40/zgemv_8f.html) para los casos de precisión simple, doble o números complejos respectivamente. Hay diferentes formas de visualizar y escribir el algoritmo de multiplicación matriz-vector. Por ejemplo para una matriz $A$ con entradas:


In [8]:
m=2
n=5
A=[[1.2]*n if i%2==0 else [1]*n for i in range(m)]

In [9]:
A

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]

In [10]:
A[0][0]

1.2

In [11]:
A[1][n-1]

1

se tiene:

### Algoritmo gaxpy *row oriented*

In [12]:
x=[2]*n
y=[0]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [13]:
y

[12.0, 10]

Si $y$ tiene valores distintos de $0$, se realiza un *update*:

In [14]:
x=[2]*n
y=[-1]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [15]:
y

[11.0, 9]

**Comentarios:**

* En la versión *row oriented* del algoritmo *gaxpy*, el **inner loop** realiza **productos punto** entre el $i$-ésimo renglón de $A$ y el vector $x$. Se realizan $m$ productos punto.

* Obsérvese que el acceso a la matriz $A$ es por renglón.

También puede escribirse al algoritmo *gaxpy* en una forma orientada por columnas:

### Algoritmo gaxpy *column oriented*

Para este algoritmo visualizamos al producto matriz-vector como una combinación lineal de las columnas de $A$:

$$Ax = \displaystyle \sum_{j=1}^n a_jx_j$$

con $a_j$ la $j$-ésima columna de $A$.


In [16]:
x=[2]*n
y=[0]*m
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

In [17]:
y

[12.0, 10]

**Obs:**

* El algoritmo de multiplicación matriz-vector (versión *row* o *column* oriented) involucra $\mathcal{O}(mn)$ operaciones o una cantidad **cuadrática** de trabajo, que podemos entender como "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $4$". Tal número de operaciones trabajan sobre una matriz o sobre una cantidad **cuadrática** de datos. A los algoritmos que realizan una cantidad cuadrática de trabajo sobre una cantidad cuadrática de datos se les cataloga de **nivel BLAS 2**. Ver [level 2](http://www.netlib.org/blas/#_level_2) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

* La versión *column oriented* se puede analizar desde el punto de vista puramente algorítmico como un intercambio entre las líneas con los índices $i$ y $j$ de cada *loop* y un acceso a los datos de la matriz por columna. O bien, se puede analizar desde el álgebra lineal indicando que el vector $y$ está en el **espacio generado** por las columnas de $A$ y cuyas coordenadas están siendo dadas por las entradas del vector $x$:

<img src="https://dl.dropboxusercontent.com/s/6a2b7rjs4a71sni/combinacion_lineal_columnas_A.png?dl=0" heigth="700" width="700">

Una ejemplo de visualización del espacio generado por las columnas de $A$, llamado **rango o imagen** de $A$, $Im(A)$, es el siguiente:

<img src="https://dl.dropboxusercontent.com/s/zkbhzv9a2jiw11b/espacio_generado_columnas_de_A.png?dl=0" heigth="400" width="400">

En este dibujo los vectores $b, r(x) \in \mathbb{R}^3$ no están en $Im(A) \subset \mathbb{R}^3$ pero $Ax$ en azul sí.

* Obsérvese que el **inner loop** de la versión *column oriented* en *gaxpy* es un **saxpy** en la que el escalar está dado por una entrada de $x$. Esto lo podemos realizar de forma explícita definiendo $A[:,j]$ a la $j$-ésima columna de $A$ por lo que $A = [A[:,1] | A[:,2] | \dots | A[:,n]]$, entonces:

```
for j=1:n
    y+=A[:,j] * x[j]
 
```

sin embargo como hemos visto, en Python con su implementación más común CPython, no es posible realizar tal indexado:

In [18]:
x=[2]*n
y=[0]*m
for j in range(n):
    y+=A[:,j]*x[j]

TypeError: list indices must be integers or slices, not tuple

a menos que incorporemos alguna paquetería que permita la **vectorización**. Un ejemplo es con [numpy](https://numpy.org/):

In [19]:
import numpy as np

In [20]:
x = 2*np.ones(n)
y = np.zeros(m)

In [21]:
x

array([2., 2., 2., 2., 2.])

In [22]:
y

array([0., 0.])

In [23]:
A

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]

In [24]:
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [25]:
A

array([[1.2, 1.2, 1.2, 1.2, 1.2],
       [1. , 1. , 1. , 1. , 1. ]])

In [26]:
for j in range(n):
    y+=A[:,j]*x[j]

In [27]:
y

array([12., 10.])

**Comentarios:**

* La vectorización como se mencionó en [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) es una herramienta para escribir programas de alto rendimiento pues incrementa el número de instrucciones por ciclo [IPC](https://en.wikipedia.org/wiki/Instructions_per_cycle) para procesadores que caen en la categoría Single Instruction Multiple Data (SIMD) de la taxonomía de Flynn (ver [liga](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)). Como ejemplo de tales procesadores están los procesadores vectoriales o en arreglo, ver [liga](https://en.wikipedia.org/wiki/Vector_processor).



**Ejemplo:**

En Unix existe la herramienta [perf](https://github.com/torvalds/linux/tree/master/tools/perf) que nos ayuda a ... otra guía útil es [liga](http://www.brendangregg.com/perf.html)

In [60]:
%%file norm_square.py
n=10**5
vector=list(range(n))
norm=0
for v in vector:
    norm+=v*v

Writing norm_square.py


In [61]:
%%bash
sudo perf stat -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

         112181115      cycles                                                        ( +-  0.34% )
         213005624      instructions              #    1.90  insn per cycle           ( +-  0.41% )
           1211284      cache-references                                              ( +-  0.16% )
             76383      cache-misses              #    6.306 % of all cache refs      ( +-  1.22% )

       0.031300940 seconds time elapsed                                          ( +-  0.77% )



**Obs:**

* Se repiten las mediciones con la *flag* `-r`.

* Se observa el número de instructiones y el número de ciclos y se reporta el IPC.

También podemos obtener las estadísticas por core:

In [62]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2             656026      cycles                                                      
S0-C0           2             221722      instructions              #    0.34  insn per cycle         
S0-C0           2               6574      cache-references                                            
S0-C0           2               4940      cache-misses              #   75.145 % of all cache refs    
S0-C1           2          114043413      cycles                                                      
S0-C1           2          221086840      instructions              #    1.94  insn per cycle         
S0-C1           2            1225916      cache-references                                            
S0-C1           2              80844      cache-misses              #    6.595 % of all cache refs    
S0-C2           2             471073      cycles                                                      
S0-C2          

In [97]:
%%bash
sudo perf stat -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

         30.996686      task-clock (msec)         #    0.994 CPUs utilized            ( +-  1.03% )
                 0      context-switches          #    0.005 K/sec                    ( +- 54.61% )
                 0      cpu-migrations            #    0.000 K/sec                  
              2075      page-faults               #    0.067 M/sec                    ( +-  0.02% )
         112080137      cycles                    #    3.616 GHz                      ( +-  0.41% )
         215514891      instructions              #    1.92  insn per cycle           ( +-  0.84% )
          45368537      branches                  # 1463.658 M/sec                    ( +-  0.97% )
            671485      branch-misses             #    1.48% of all branches          ( +-  0.44% )

       0.031193583 seconds time elapsed                                          ( +-  1.03% )



In [101]:
%%bash
sudo perf stat -a --per-core -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2          58.798485      cpu-clock (msec)          #    1.895 CPUs utilized          
S0-C0           2                  3      context-switches          #    0.051 K/sec                  
S0-C0           2                  0      cpu-migrations            #    0.000 K/sec                  
S0-C0           2               2074      page-faults               #    0.035 M/sec                  
S0-C0           2          110675585      cycles                    #    1.882 GHz                    
S0-C0           2          210018815      instructions              #    1.90  insn per cycle         
S0-C0           2           44016437      branches                  #  748.598 M/sec                  
S0-C0           2             668391      branch-misses             #    1.52% of all branches        
S0-C1           2          58.854613      cpu-clock (msec)          #    1.896 CPUs utilized          
S0-C1          

Utilizando numpy:

In [91]:
%%file norm_square_numpy.py
import numpy as np
n=10**5
vector=np.arange(n)
vector@vector

Overwriting norm_square_numpy.py


In [92]:
%%bash
sudo perf stat -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

        2506788309      cycles                                                        ( +-  0.30% )
        1287968310      instructions              #    0.51  insn per cycle           ( +-  0.19% )
           7584990      cache-references                                              ( +-  0.51% )
            418690      cache-misses              #    5.520 % of all cache refs      ( +-  1.66% )

       0.140431634 seconds time elapsed                                          ( +-  0.44% )



In [93]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2          569083396      cycles                                                      
S0-C0           2          221162194      instructions              #    0.39  insn per cycle         
S0-C0           2              41375      cache-references                                            
S0-C0           2              10728      cache-misses              #   25.929 % of all cache refs    
S0-C1           2          818648862      cycles                                                      
S0-C1           2          622617258      instructions              #    0.76  insn per cycle         
S0-C1           2            7603960      cache-references                                            
S0-C1           2             394100      cache-misses              #    5.183 % of all cache refs    
S0-C2           2          570103322      cycles                                                      
S0-C2          

In [98]:
%%bash
sudo perf stat -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

        659.040707      task-clock (msec)         #    4.659 CPUs utilized            ( +-  0.49% )
              3690      context-switches          #    0.006 M/sec                    ( +- 71.45% )
                 1      cpu-migrations            #    0.002 K/sec                    ( +- 15.02% )
              4753      page-faults               #    0.007 M/sec                    ( +-  0.04% )
        2506584507      cycles                    #    3.803 GHz                      ( +-  0.48% )
        1289772392      instructions              #    0.51  insn per cycle           ( +-  0.09% )
         257070416      branches                  #  390.068 M/sec                    ( +-  0.12% )
           6516911      branch-misses             #    2.54% of all branches          ( +-  0.10% )

       0.141442430 seconds time elapsed                                          ( +-  0.29% )



In [100]:
%%bash
sudo perf stat -a --per-core -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

S0-C0           2         276.867110      cpu-clock (msec)          #    1.967 CPUs utilized          
S0-C0           2                 16      context-switches          #    0.058 K/sec                  
S0-C0           2                  1      cpu-migrations            #    0.004 K/sec                  
S0-C0           2                  4      page-faults               #    0.014 K/sec                  
S0-C0           2          568674514      cycles                    #    2.054 GHz                    
S0-C0           2          221738525      instructions              #    0.39  insn per cycle         
S0-C0           2           41527823      branches                  #  149.992 M/sec                  
S0-C0           2             755939      branch-misses             #    1.82% of all branches        
S0-C1           2         276.885651      cpu-clock (msec)          #    1.967 CPUs utilized          
S0-C1          

Para el caso del producto *gaxpy column oriented*

In [241]:
np.random.seed(2020)
m=10**3
n=10**4
A=np.random.rand(m,n)
file='A.txt'
np.savetxt(file,A)

In [255]:
%%file mult_matrix_vector.py
m=10**3
n=10**4
x=[2]*n
y=[0]*m
A = []
file='A.txt'
with open(file,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])      
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

Overwriting mult_matrix_vector.py


In [243]:
%%file mult_matrix_vector_numpy.py
import numpy as np
m=10**3
n=10**4
x = 2*np.ones(n)
y = np.zeros(m)
file='A.txt'
A = np.loadtxt(file)
for j in np.arange(n):
    y+=A[:,j]*x[j]

Overwriting mult_matrix_vector_numpy.py


In [256]:
%%bash
sudo perf stat -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_vector.py


 Performance counter stats for 'python3 mult_matrix_vector.py' (5 runs):

       24447699995      cycles                                                        ( +-  0.66% )
       60499712413      instructions              #    2.47  insn per cycle           ( +-  0.76% )
          92005195      cache-references                                              ( +-  0.31% )
          10602296      cache-misses              #   11.524 % of all cache refs      ( +-  0.16% )

       6.136940697 seconds time elapsed                                          ( +-  0.68% )



In [257]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_vector.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2           24097137      cycles                                                      
S0-C0           2            3670739      instructions              #    0.15  insn per cycle         
S0-C0           2             359628      cache-references                                            
S0-C0           2             180474      cache-misses              #   50.184 % of all cache refs    
S0-C1           2          148923456      cycles                                                      
S0-C1           2           99920603      instructions              #    0.67  insn per cycle         
S0-C1           2            1040108      cache-references                                            
S0-C1           2             566167      cache-misses              #   54.433 % of all cache refs    
S0-C2           2        25320104931      cycles                                                      
S0-C2           

In [246]:
%%bash
sudo perf stat -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy.py' (5 runs):

       26226110128      cycles                                                        ( +-  0.26% )
       61338384935      instructions              #    2.34  insn per cycle           ( +-  0.31% )
          61590832      cache-references                                              ( +-  0.19% )
           8303525      cache-misses              #   13.482 % of all cache refs      ( +-  1.00% )

       6.077492905 seconds time elapsed                                          ( +-  0.28% )



In [248]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2          704131375      cycles                                                      
S0-C0           2          321146309      instructions              #    0.46  insn per cycle         
S0-C0           2             890166      cache-references                                            
S0-C0           2             517658      cache-misses              #   58.153 % of all cache refs    
S0-C1           2          837185551      cycles                                                      
S0-C1           2          283408858      instructions              #    0.34  insn per cycle         
S0-C1           2             511169      cache-references                                            
S0-C1           2             263779      cache-misses              #   51.603 % of all cache refs    
S0-C2           2          660961714      cycles                                                      
S0-C2           

* El algoritmo *gaxpy row oriented* puede escribirse de forma más compacta haciendo uso de la definición de producto punto estándar: $x^Ty$ para dos vectores columna $x$ y $y$. En el caso de una matriz $A$ se tiene:

```
for i=1:m
    y[i]+=A[i,:]^T*x
```

y en Python:

In [76]:
x = 2*np.ones(n)
y = np.zeros(m)
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [81]:
for i in range(m):
    y[i]+=A[i,:].dot(x)

In [82]:
y

array([12., 10.])

en donde se utilizó la función [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) de *numpy*.

* 

Matriz matriz

In [282]:
np.random.seed(2020)
m=10**2
r=10**3

A=np.random.rand(m,r)
fileA='A.txt'
np.savetxt(fileA,A)

In [283]:
np.random.seed(2021)
r=10**3
n=10**2

B=np.random.rand(r,n)
fileB='B.txt'
np.savetxt(fileB,B)

In [284]:
%%file mult_matrix_matrix.py
m=10**2
r=10**3
n=10**2

A = []
B = []
fileA='A.txt'
fileB='B.txt'

with open(fileA,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')]) 
        
with open(fileB,'r') as f:
    for l in f:
        B.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])  

C=[[0]*n for i in range(m)]

for i in range(m):
    for j in range(n):
        for k in range(r):
            C[i][j]+=A[i][k]*B[k][j]


Overwriting mult_matrix_matrix.py


In [301]:
%%file mult_matrix_matrix_numpy.py
import numpy as np
m=10**2
r=10**3
n=10**2

fileA='A.txt'
fileB='B.txt'
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)
C = A@B

Overwriting mult_matrix_matrix_numpy.py


In [302]:
%%bash
sudo perf stat  -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2         8700209007      cycles                                                        (49.90%)
S0-C0           2        23022311113      instructions              #    2.65  insn per cycle           (49.72%)
S0-C0           2           39143700      cache-references                                              (49.62%)
S0-C0           2            4888264      cache-misses              #   12.488 % of all cache refs      (49.62%)
S0-C1           2          138757902      cycles                                                        (49.90%)
S0-C1           2           58136635      instructions              #    0.42  insn per cycle           (49.72%)
S0-C1           2            2801700      cache-references                                              (49.62%)
S0-C1           2             758476      cache-misses              #   27.072 % of all cache refs      (49.62%)
S0-C2           2         8613646213   

In [308]:
%%bash
sudo perf stat -a -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix.py


 Performance counter stats for 'system wide' (5 runs):

       26042847565      cycles                                                        ( +-  1.21% )  (50.05%)
       54311252425      instructions              #    2.09  insn per cycle           ( +-  0.74% )  (49.93%)
         116077909      cache-references                                              ( +-  0.72% )  (49.83%)
          44564342      cache-misses              #   38.392 % of all cache refs      ( +-  2.07% )  (49.71%)

       2.215476995 seconds time elapsed                                          ( +-  1.11% )



In [309]:
%%bash
sudo perf stat -a -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (5 runs):

        4795799420      cycles                                                        ( +-  1.54% )  (50.44%)
        5686535426      instructions              #    1.19  insn per cycle           ( +-  2.80% )  (49.59%)
          17280949      cache-references                                              ( +-  1.34% )  (48.89%)
           5576638      cache-misses              #   32.270 % of all cache refs      ( +-  3.00% )  (48.17%)

       0.287860114 seconds time elapsed                                          ( +-  2.30% )



In [303]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2         1398564453      cycles                                                        (49.96%)
S0-C0           2         1903511582      instructions              #    1.36  insn per cycle           (48.68%)
S0-C0           2            6476427      cache-references                                              (47.38%)
S0-C0           2             661864      cache-misses              #   10.220 % of all cache refs      (47.22%)
S0-C1           2          848956640      cycles                                                        (49.98%)
S0-C1           2          471382739      instructions              #    0.56  insn per cycle           (48.68%)
S0-C1           2            2761621      cache-references                                              (47.39%)
S0-C1           2             310476      cache-misses              #   11.243 % of all cache refs      (47.21%)
S0-C2           2         1435944253   

In [304]:
%%file mult_matrix_matrix_numpy_dot_product.py
import numpy as np
m=10**2
r=10**3
n=10**2

fileA='A.txt'
fileB='B.txt'
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)
C = np.zeros((m,n))
for i in np.arange(m):
        for j in np.arange(n):
                C[i][j]+= A[i,:].dot(B[:,j])

Overwriting mult_matrix_matrix_numpy_dot_product.py


In [307]:
%%bash
sudo perf stat -a -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (5 runs):

        4814719842      cycles                                                        ( +-  0.43% )  (49.52%)
        5906198829      instructions              #    1.23  insn per cycle           ( +-  0.36% )  (48.46%)
          18627898      cache-references                                              ( +-  1.11% )  (47.98%)
           5888760      cache-misses              #   31.613 % of all cache refs      ( +-  0.45% )  (47.80%)

       0.301519329 seconds time elapsed                                          ( +-  0.81% )



In [305]:
%%bash
sudo perf stat -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2         1486360022      cycles                                                        (50.78%)
S0-C0           2         2026044472      instructions              #    1.36  insn per cycle           (49.53%)
S0-C0           2            9884274      cache-references                                              (48.27%)
S0-C0           2             927317      cache-misses              #    9.382 % of all cache refs      (47.02%)
S0-C1           2          569145121      cycles                                                        (50.77%)
S0-C1           2          249699694      instructions              #    0.44  insn per cycle           (49.52%)
S0-C1           2             314017      cache-references                                              (48.27%)
S0-C1           2             129114      cache-misses              #   41.117 % of all cache refs      (47.01%)
S0-C2           2         1502391734   

**Referencias:**

* E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

* G. H. Golub, C. F. Van Loan, Matrix Computations, John Hopkins University
Press, 2013.

* [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb)